# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="./Tennis_Windows_x86_64/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
# for i in range(1, 6):                                      # play game for 5 episodes
#     env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
#     states = env_info.vector_observations                  # get the current state (for each agent)
#     scores = np.zeros(num_agents)                          # initialize the score (for each agent)
#     while True:
#         actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#         actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#         env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#         next_states = env_info.vector_observations         # get next state (for each agent)
#         rewards = env_info.rewards                         # get reward (for each agent)
#         dones = env_info.local_done                        # see if episode finished
#         scores += env_info.rewards                         # update the score (for each agent)
#         states = next_states                               # roll over states to next time step
#         if np.any(dones):                                  # exit loop if episode finished
#             break
#     print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))

When finished, you can close the environment.

In [6]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [7]:
# from ddpg_agent import Agent
from ddpg_agent import Agent
import os
import regex
from collections import deque
import torch
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
def save_file(project_name, ext="pth"):
    file_number = []
    for file in os.listdir():
        if regex.match(r"{}".format("{}.*.{}".format(project_name, ext)), file):
            extract = regex.search(r'[0-9][0-9]', file)
            if file[extract.span()[0]] == "0":
                file_number.append(int(file[extract.span()[1]-1]))
            else:
                file_number.append(int("{}{}".format(file[extract.span()[0]],file[extract.span()[1]-1])))
            
            file_number.sort()
            file_number.reverse()
#             print(file_number)
            
            if file_number[0] < 9:
                file_name = "{}_0{}.{}".format(project_name, file_number[0]+1, ext)
            else:
                file_name = "{}_{}.{}".format(project_name, file_number[0]+1, ext)

    if len(file_number) == 0:
        file_name = "{}_00.{}".format(project_name, ext)
    
    print("Latest file name should be {}".format(file_name))
    return file_name

In [9]:
seed = 1 # Seed for random control
agent = Agent(state_size, action_size, seed, num_agents)

In [10]:
def ddpg(n_episodes=50000, max_t=1000):
    
    scores_deque = deque(maxlen=100)
    scores = []
    BATCH_SIZE = 128
    actor_pth = save_file("p3_collab_actor", ext="pth")
    critic_pth = save_file("p3_collab_critic", ext="pth")
    scored_hit = False
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
        states = env_info.vector_observations                  # get the current state (for each agent)
        agent.reset()
        score = np.zeros(num_agents)                           # initialize the score (for each agent)
        for t in range(max_t):
#         while True:
            actions = agent.act(states)                        # select an action (for each agent)
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            agent.step(states, actions, rewards, next_states, dones, t)
            score += env_info.rewards                          # update the score (for each agent)
            states = next_states                               # roll over states to next time step
            if np.any(dones):                                  # exit loop if episode finished
                break
        
        avg_scores = np.mean(score)
        scores_deque.append(avg_scores)
        scores.append(avg_scores)
        
        if scored_hit == False:
            print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, np.mean(scores_deque), np.mean(score)), end="")
            if i_episode % 50 == 0:
                print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, np.mean(scores_deque), np.mean(score)))
            
        elif avg_scores >= 0.5 and scored_hit == False:
            print('\rThe CC has reached 30 avg score at Episode {} with Average Score: {:.2f}'.format(i_episode, np.mean(scores_deque)))
            scored_hit == True
            
        else:
            print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, np.mean(scores_deque), np.mean(score)), end="")
            if i_episode % 10 == 0:
                print('\rEpisode {}\tAverage Score: {:.2f}\tScore: {:.2f}'.format(i_episode, np.mean(scores_deque), np.mean(score)))
                
        if np.mean(scores_deque) >= 0.5 and len(scores_deque) == 100 and min(scores_deque) >=0.5:
            print('\rThe CC is solved at Episode {} with Average Score: {:.2f}!'.format(i_episode, np.mean(scores_deque)))
            torch.save(agent.actor_local.state_dict(), actor_pth)
            torch.save(agent.critic_local.state_dict(), critic_pth)
            
            # Save the scores
            fo = open(save_file("p3_collab_score", ext = "txt"), "w")
            fo.writelines(str(scores))
            fo.close()

            fig = plt.figure()
            ax = fig.add_subplot(111)
            plt.plot(np.arange(1, len(scores)+1), scores)
            plt.ylabel('Score')
            plt.xlabel('Episode #')
            plt.savefig('{}'.format(save_file("p3_collab_score_plt", ext="png")))
            plt.show()

            return scores
        
    # At the end of training with no luck
    print("No luck in solving the CC problem with {} episode".format(n_episodes))
    return scores

In [11]:
scores = ddpg()

Latest file name should be p3_collab_actor_00.pth
Latest file name should be p3_collab_critic_00.pth
Episode 1	Average Score: -0.00	Score: -0.00

C:\Users\root\AppData\Local\conda\conda\envs\drlnd\lib\site-packages\torch\nn\functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Episode 2	Average Score: -0.00	Score: -0.00

C:\Users\root\udacity\p3_collab-compet\ddpg_agent.py:129: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(self.critic_local.parameters(), 1.0) #clip the gradient for the critic network (Udacity hint)


Episode 50	Average Score: 0.00	Score: -0.000
Episode 100	Average Score: -0.00	Score: -0.00
Episode 150	Average Score: -0.00	Score: -0.00
Episode 200	Average Score: 0.00	Score: -0.000
Episode 250	Average Score: 0.01	Score: -0.00
Episode 300	Average Score: 0.01	Score: 0.050
Episode 350	Average Score: 0.06	Score: 0.050
Episode 393	Average Score: 0.46	Score: 2.600

KeyboardInterrupt: 

In [12]:
env_info = env.reset(train_mode=False)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = agent.act(states)                        # select an action (for each agent)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 2.600000038743019
